In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("max_rows", 100)

In [3]:
# Read data files
df = pd.read_excel('data/KPMG Data_Spend Analytics 2.xlsx')
df_dd = pd.read_excel('data/DataDictionary_Spend Analytics.xlsx')

In [4]:
df.shape,df_dd.shape

((75349, 65), (64, 4))

# Write an output file with missing information for the report

In [5]:
dt = (df.isna().mean().round(4) * 100).reset_index()
dt.columns = ['Column Name','% Null Values']

In [6]:
dt['% Zero Values'] = (df==0).sum(axis=0).values
dt['% Zero Values'] = round((dt['% Zero Values'] / df.shape[0]),2)

In [7]:
dt['Num Unique Values'] = df.nunique().values

In [8]:
dt['DataType'] = df.dtypes.values

In [9]:
dt.head()

,Column Name,% Null Values,% Zero Values,Num Unique Values,DataType
0,Purch.Doc.,0.00,0.0,42596,int64
1,Item,0.00,0.0,42,int64
2,Changed On,0.00,0.0,210,datetime64[ns]
3,Short Text,0.00,0.0,7332,object
4,Material,2.12,0.0,2757,float64


In [10]:
# Get the column description 
# dt = dt.merge(df_dd[['Cloumn Name','Description']], on='Cloumn Name', how='left')

In [11]:
# Write the output file 
dt.to_excel('preprocessing/SA_about_data.xlsx', index=False)

In [12]:
# Check for duplicates
df.duplicated().sum()

0

In [13]:
# Check for date spread 

In [14]:
df['Changed On'] =  pd.to_datetime(df['Changed On'], format='%Y-%m-%d')
df['Order_yr'] = df['Changed On'].dt.year
df['Order_mnth'] = df['Changed On'].dt.month

In [15]:
dt = pd.pivot_table(df,index=['Order_yr'],values=['Purch.Doc.'],columns=['Order_mnth'],
               aggfunc=pd.Series.nunique,fill_value=0,margins=True)
dt.reset_index()
dt

Purch.Doc.                                                        
Order_mnth          1      2      3    4  5  6  7  8   9  10  11    12    All
Order_yr                                                                     
2012                0      0      0    0  0  0  1  0   0   0   0     0      1
2014                0      0      0    0  0  1  0  2   0   0   0     2      5
2015                0      1      0    0  0  0  0  0   0   0   0     0      1
2017                0      0      0    0  3  0  2  0   1   2   0     3     11
2018                2      1      1    4  2  5  3  7  16  29  71  2895   3034
2019            14931  12454  12134  168  0  0  0  0   0   0   0     0  39582
All             14933  12456  12135  172  5  6  6  9  17  31  71  2900  42596

In [16]:
# Write df for the report 
dt.to_excel('preprocessing/SA_YrWiseSpread.xlsx') #,index=False)

# Preprocess data 

# 1. Keep only 2018 and 2019 data

In [17]:
df.shape

(75349, 67)

In [18]:
df = df[df['Order_yr'].isin([2018,2019])]
df.shape

(75295, 67)

# 2. Remove more than 90# NAN columns

In [19]:
# Prints only nan columns
df[df.columns[df.isnull().any()]].isnull().mean().round(4) * 100

Material                2.12
SLoc                   17.02
TrackingNo             93.46
DCI                    10.26
A                      95.16
Cns                    95.16
BUn                     2.12
Price Date              0.37
Un                     59.96
S                      97.29
Profit Ctr              2.12
S.1                    99.20
MTyp                    2.12
NCM Code                2.49
Spec. Stk Valuation    97.29
Requested By           91.86
Priority               80.58
Input Tax Credit       79.27
dtype: float64

In [20]:
# Remove columns with more than 90% null...
df.drop(['TrackingNo','A','Cns','S','S.1','Spec. Stk Valuation','Requested By'], inplace=True, axis='columns')
df.shape

(75295, 60)

# 3. Check for unique values

In [21]:
df.nunique()

Purch.Doc.                             42579
Item                                      42
Changed On                               193
Short Text                              7304
Material                                2751
CoCd                                       5
Plnt                                     115
SLoc                                    1108
Matl Group                               145
PO Quantity                            14982
OUn                                       31
OPU                                       31
Conv.                                      1
Eq. To                                     2
Net Price                               8709
Per                                       89
Net Value                              36638
Gross value                            36024
GRT                                        1
Overdel. Tol.                             15
Underdel.Tol.                              6
DCI                                        1
Item.1    

In [22]:
# Remove columns with only 1 unique value
cols = ['Conv.', 'GRT', 'DCI', 'Agr. Cum. Qty', 'TOZ', 'Quantity', 'Cat',
       'Net value', 'Object no.', 'Time of Transmission',
       'Next Transmission Number', 'Itm', 'Itm.1', 'Requirement Urgency',
       'CRM  Item No', 'Down Payment Amount', 'Item.2']

df.drop(cols, inplace=True, axis='columns')
df.shape

(75295, 43)

In [23]:
# Unit of Measure and weight/volumme,shelf life , used for storage analytics
#cols = ['OUn','OPU','BUn','Un','Gross Weight','Volume','Net Weight','RShLi']
#df.drop(cols, inplace=True, axis='columns')
#df.shape

In [24]:
#cols = ['Item.1','Eq. To','Non-deductible','Input Tax Credit','Reb.basis']
#df.drop(cols, inplace=True, axis='columns')
#df.shape

In [25]:
#cols = ['Ordered By','Approved By','Indenter ID']
#df.drop(cols, inplace=True, axis='columns')
#df.shape

# 4 Preprocess null values 

In [26]:
# Prints only nan columns
df[df.columns[df.isnull().any()]].isnull().mean().round(4) * 100

Material             2.12
SLoc                17.02
BUn                  2.12
Price Date           0.37
Un                  59.96
Profit Ctr           2.12
MTyp                 2.12
NCM Code             2.49
Priority            80.58
Input Tax Credit    79.27
dtype: float64

# 1. Process Material

In [27]:
# NAN Materials are 'Services' - from  short text and Material Group
# Create a new code for Services
# Fill Material NAN with with 8900000.0
df['Material'] = df['Material'].fillna(value=8900000.0)

# 2. SLoc,Profit Ctr,MTyp,NCM Code

In [28]:
# Fill NAN with groupby CoCd mode

df['SLoc'] = df.groupby('CoCd')['SLoc'].apply(lambda x: x.fillna(x.value_counts().index[0]))
df['Profit Ctr'] = df.groupby('CoCd')['Profit Ctr'].apply(lambda x: x.fillna(x.value_counts().index[0]))
df['MTyp'] = df.groupby('CoCd')['MTyp'].apply(lambda x: x.fillna(x.value_counts().index[0]))
df['NCM Code'] = df.groupby('CoCd')['NCM Code'].apply(lambda x: x.fillna(x.value_counts().index[0]))

# 3. Process Priority

In [29]:
df['Priority'].value_counts(dropna=False)

NaN          60672
Medium        7102
High          2683
Low           2424
Very High     2307
MEDIUM          41
MEDIAM          36
medium          28
high             1
HIGH             1
Name: Priority, dtype: int64

In [30]:
# First clean the mutiplicity of the categories
# Keep Low, Medium,High,Very High only

df['Priority'] = np.where(df['Priority'].isin(['MEDIUM','MEDIAM','medium']),'Medium', df['Priority'])
df['Priority'] = np.where(df['Priority'].isin(['high','HIGH']),'High', df['Priority'])

In [31]:
df['Priority'].value_counts(dropna=False)

NaN          60672
Medium        7207
High          2685
Low           2424
Very High     2307
Name: Priority, dtype: int64

In [32]:
pd.crosstab(df['Priority (Material Required Within)'].fillna('missing'),df['Priority'].fillna('missing'), margins=True)

Priority,High,Low,Medium,Very High,missing,All
Priority (Material Required Within),,,,,,
0,0,0,22,0,60670,60692
1,1939,1,0,48,0,1988
2,642,0,1,2259,2,2904
3,48,0,5,0,0,53
4,0,0,1159,0,0,1159
5,2,2,2247,0,0,2251
6,0,0,1011,0,0,1011
7,0,831,84,0,0,915
8,0,6,37,0,0,43


In [33]:
# Fill Priority NAN with with Normal
df['Priority'] = df['Priority'].fillna(value='Normal')

In [34]:
df['Priority'].value_counts(normalize=True,dropna=False)

Normal       0.805791
Medium       0.095717
High         0.035660
Low          0.032193
Very High    0.030639
Name: Priority, dtype: float64

# 4. Process Price Date

In [35]:
# # Sort the data by Materail and Changed Date ( Order Date) and forward fill the price date.
# df1 = df.sort_values(by=['Material', 'Changed On'],ascending = [True, True])
# # fill the missing values
# df1['Price Date'] = df1['Price Date'].ffill()

In [36]:
# Fill  it by Changed Date
df['Price Date'].fillna(df['Changed On'], inplace=True)

# Important features are Gross Value, Net price , Material, Material Group, Short Text
# Check on these 

In [37]:
df['Gross value'].describe()

count    7.529500e+04
mean     3.182084e+05
std      2.094363e+06
min      0.000000e+00
25%      6.736700e+02
50%      4.560000e+04
75%      2.238600e+05
max      2.120000e+08
Name: Gross value, dtype: float64

In [38]:
# How any zeroes Gross Value
df[df['Gross value']==0].shape

(13425, 43)

In [39]:
df[df['Gross value']==0]['Order_yr'].value_counts()

2019    12124
2018     1301
Name: Order_yr, dtype: int64

# 5 Remove 0 Gross Value

In [40]:
df.shape

(75295, 43)

In [41]:
df = df[df['Gross value']!=0]
df.shape

(61870, 43)

In [42]:
# Write the cleaned file 
df.to_csv("preprocessing/SA_clean.csv", index=False)